In [ ]:
!pip install selenium
!pip install pymystem3
!pip install wordcloud
!pip install scikit-learn
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

!dpkg -i google-chrome-stable_current_amd64.deb

!apt-get -f -y install
!apt-get update
!apt-get install -y chromium-chromedriver
!apt-get install -y google-chrome-stable
!pip install webdriver_manager

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 29.6 MB/s eta 0:00:00
--2025-12-20 13:24:31--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 142.251.107.136, 142.251.107.91, 142.251.107.190, ...
Connecting to dl.google.com (dl.google.com)|142.251.107.136|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 117840328 (112M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 112.38M   401MB/s    in 0.3s    

2025-12-20 13:24:31 (401 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [117840328/117840328]

Selecting previously unselected package google-chrome-stable.
(Reading database ... 117528 files and directories currently installed.)
Preparing to unpack google-chrome-stable_current_amd64.deb ...
Unpacking google-chrome-stabl

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
# from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from pymystem3 import Mystem
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
import time
import requests
import pandas as pd
import nltk
import re
import matplotlib.pyplot as plt
from datetime import datetime

options = webdriver.ChromeOptions()
options.add_argument("--ignore-certificate-errors")
options.add_argument("--allow-insecure-localhost")
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(options=options)

# На всякий случай оставила, потом уберу
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

# проверка перед заупском (подкючается или нет)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
# Стягиваем тайтл сайта
driver.get('https://www.riatomsk.ru/novosti')
print(driver.title)

# Функции пошли

def is_weather_related(title, url="", lead=""):
    text = f"{title} {lead}".lower().strip()

    # 100 и одно регулярное для НЕПОГОДЫ
    non_weather_contexts = [
        r'ледов(ая|ом|ых)\s+(городок|переправ|каток|горка|арена|трасс|шоу|спектакль|праздник|фестивал|развлечени)',
        r'зимн(ий|его)\s+(каток|городок|лагерь|фестивал)',
        r'мерзнуть\s+культурно',
        r'(постро|стро|сооруд|открыт|открыли|открыта)\s+ледов(ый|ая|ое|ых)',
        r'лед\s+для\s+(развлечений|аттракционов|шоу|праздника)',
        r'лёд\s+в\s+центре\s+города',
        r'лед\s+на\s+(площад|набережн|центре)',
        r'(холод|мороз)\s+в\s+названи(и|е)\s+(праздник|фестивал|акци|мероприят)',
        r'морозн(ый|ая)\s+(фестив|ярмарк|городок|шоу|десерт)',
        r'(льд|лед)\s+используют\s+для',
        r'лед\s+в\s+(целях|для)\s+(развлечени|строительств|съёмок)',
        r'(снежн|ледян)\s+фигур',
        r'новогодн(ий|ая|ое)\s+(городок|ярмарк|праздник)',
        r'праздник\s+(зим|новый\s+год|рождеств|ханук|снеж)',
        r'лед\s+в\s+рамк(ах|и)\s+(праздник|фестивал)',
        r'(морозко|дед\s+мороз|снегурочк|новый\s+год|рождеств|ханук|святки)'
    ]

    for pattern in non_weather_contexts:
        if re.search(pattern, text, re.IGNORECASE):
            return False

    explicit_keywords = [
        'погода', 'прогноз', 'метео', 'гидромет', 'синоптик', 'осадки', 'ветер',
        'градус', 'температур', 'циклон', 'антициклон', 'давление',
        'дождь', 'ливень', 'снег', 'метель', 'гололед', 'гололедица',
        'похолодание', 'потепление', 'мороз', 'жара', 'холод', 'ветрено',
        'туман', 'град', 'пасмурно', 'облачно', 'ясно'
    ]
    if any(kw in text for kw in explicit_keywords):
        return True

    # Тысяча и одно регулярное для погоды
    forecast_patterns = [
        r'ожида[ею]т[ся]*\s+(снег|дождь|ливень|ветер|минус|мороз|метель|гололед|осадки|туман|пыль|град|холод|тепло)',
        r'(снег|дождь|ветер|мороз|минус|холод|тепло|метель)\s+ожида[ею]т[ся]*',

        r'установится\s+(мороз|холод|тепло|ветер|ясно|пасмурно)',
        r'сохранится\s+(мороз|холод|ветреная погода|снег|дождь|осадки)',
        r'при[йё]дет\s+(холод|мороз|циклон|потепление|ветер)',
        r'усилится\s+(ветер|снегопад|мороз|дождь)',

        r'(в\s+(ближайшие\s+дни|выходные|субботу|воскресенье|понедельник|ночью|днем|утром))\s+.*?(снег|дождь|ветер|мороз|минус|тепло|осадк)',
        r'(снег|дождь|ветер|мороз|холод|тепло)\s+.*?(в\s+(выходные|воскресенье|ночью|днем|утром))',

        r'до\s+минус\s*\d{1,2}',
        r'до\s+-\d{1,2}',
        r'до\s+\+\d{1,2}',
        r'свыше\s+\+\d{1,2}',
        r'свыше\s+минус\s*\d{1,2}',
        r'ниже\s+\+\d{1,2}',
        r'ниже\s+минус\s*\d{1,2}',
        r'холод\s+до\s+\d{1,2}',

        r'порыв(ы|ами)?\s+ветр(а|ом)\s+до\s+\d{1,2}',

        r'(в\s+(понедельник|вторник|среду|четверг|пятницу|субботу|воскресенье|выходные)).*?(тепл|минус|осадк|дожд|снег|ветер|мороз)'
    ]

    for pattern in forecast_patterns:
        if re.search(pattern, text, re.IGNORECASE):
            return True

    url_lower = url.lower()
    if any(kw in url_lower for kw in ['pogoda', 'prognoz', 'weather', 'meteo', 'gismeteo']):
        return True

    return False

# Теги, лиды и прочие интересности
def get_article_details(url, session):
    try:
        resp = session.get(url, timeout=10)
        resp.raise_for_status()
        soup = BeautifulSoup(resp.content, 'lxml')

        date_elem = soup.find('time')
        pub_date = date_elem['datetime'] if date_elem and date_elem.has_attr('datetime') else None

        if pub_date:
            try:
                dt = datetime.fromisoformat(pub_date.replace('Z', '+00:00'))
                if dt.year != 2025:
                    return None
            except:
                pass

        lead = ''
        body = soup.find('div', class_=lambda x: x and 'article-body' in x)
        if body:
            first_p = body.find('p')
            lead = first_p.get_text(strip=True) if first_p else ''

        paragraphs = body.find_all('p') if body else []
        full_text = '\n'.join(p.get_text(strip=True) for p in paragraphs)

        tags = []
        tag_containers = soup.find_all(['div', 'ul', 'span'], class_=lambda x: x and ('tag' in x.lower() or 'topic' in x.lower()))
        for container in tag_containers:
            for tag in container.find_all(['a', 'span']):
                t = tag.get_text(strip=True)
                if t and len(t) > 1:
                    tags.append(t)
        tags = list(set(tags))

        return {
            'date': pub_date,
            'lead': lead,
            'text': full_text,
            'tags': tags
        }
    except Exception as e:
        print(f"Ошибка при парсинге статьи {url}: {e}")
        return None

# Для даты
from dateutil import parser as date_parser

def extract_pub_date(soup, url):
    time_tag = soup.find('time', datetime=True)
    if time_tag and time_tag.get('datetime'):
        dt_str = time_tag['datetime'].strip()
        try:
            return date_parser.isoparse(dt_str.replace('Z', '+00:00')).isoformat()
        except:
            pass

    date_candidates = soup.find_all(['p', 'div', 'span'], string=re.compile(r'\d{1,2}\s+[а-яА-ЯёЁ]+\s+2025'))
    for cand in date_candidates:
        text = cand.get_text()
        try:
            dt = date_parser.parse(text, languages=['ru'], dayfirst=True)
            if dt.year == 2025:
                return dt.isoformat()
        except:
            continue

    m = re.search(r'/(\d{4})/(\d{2})/(\d{2})/', url)
    if m:
        y, mth, d = m.groups()
        if y == '2025':
            return f"{y}-{mth}-{d}T00:00:00"

    return None

import time
import json
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Чтобы парсилось только до 1 января 2025
CUTOFF_DATE = datetime(2025, 1, 1)

seen_links = set()
articles_data = []
last_known_block_count = 0

# Создаём драйвер
def create_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return driver, WebDriverWait(driver, 15)

driver, wait = create_driver()

driver.get("https://www.riatomsk.ru/novosti")

# Закрываем банер куки, а то дальше ничего не получится
try:
    btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".cookie-agreement__button")))
    driver.execute_script("arguments[0].click();", btn)
    time.sleep(0.6)
except:
    pass

# Для обработки блоков
def collect_articles_from_blocks(blocks, stop_on_pre_2025=False):
    added = 0
    earliest_in_batch = None
    early_stop = False

    for block in blocks:
        try:
            link_elem = block.find_element(By.CLASS_NAME, "rubric-article__link")
            link = link_elem.get_attribute("href")
            if not link or link in seen_links:
                continue

            title_elem = block.find_element(By.CLASS_NAME, "rubric-article__title")
            title = title_elem.text.strip()
            if not title:
                continue

            try:
                time_el = block.find_element(By.TAG_NAME, "time")
                date_str = time_el.get_attribute("datetime")
                pub_date = datetime.fromisoformat(date_str[:10])
            except:
                pub_date = None

            if pub_date:
                if earliest_in_batch is None or pub_date < earliest_in_batch:
                    earliest_in_batch = pub_date

            seen_links.add(link)

            if pub_date and pub_date < CUTOFF_DATE:
                # Для ранней остановки
                if stop_on_pre_2025:
                    early_stop = True
                    break
                continue

            # Сохранение только для 2025
            if pub_date and pub_date >= CUTOFF_DATE:
                articles_data.append({
                    "title": title,
                    "link": link,
                    "date": date_str[:10]
                })
                added += 1

        except Exception:
            continue

    return added, earliest_in_batch, early_stop

# Собираем первые 20 статей без выкрутасов
all_blocks = driver.find_elements(By.CLASS_NAME, "rubric-article")
new_blocks = all_blocks[last_known_block_count:]
added, _, _ = collect_articles_from_blocks(new_blocks)
last_known_block_count = len(all_blocks)

# Нажимаем на кнопку "показать ещё"
try:
    load_more = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, ".get-more a.button.button_block")))
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", load_more)
    time.sleep(1)
    driver.execute_script("arguments[0].click();", load_more)
    time.sleep(0.7)
    # Сбор после клика
    all_blocks = driver.find_elements(By.CLASS_NAME, "rubric-article")
    new_blocks = all_blocks[last_known_block_count:]
    added, _, _ = collect_articles_from_blocks(new_blocks)
    last_known_block_count = len(all_blocks)
except Exception as e:
    print("Не удалось кликнуть 'Показать ещё':", e)

# Бесконечно крутим (с одним но, но это дальше)
no_new = 0
total_scrolls = 0
max_scrolls = 200
max_no_new = 5

while total_scrolls < max_scrolls and no_new < max_no_new:
    last_height = driver.execute_script("return document.body.scrollHeight")
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(0.7)

    try:
        WebDriverWait(driver, 8).until(
            lambda d: len(d.find_elements(By.CLASS_NAME, "rubric-article")) > last_known_block_count
        )
    except:
        no_new += 1
        print(f"→ Нет новых блоков ({no_new}/{max_no_new})")
        continue

    all_blocks = driver.find_elements(By.CLASS_NAME, "rubric-article")
    new_blocks = all_blocks[last_known_block_count:]
    last_known_block_count = len(all_blocks)

    # print(f"Новых блоков: {len(new_blocks)}")
    if not new_blocks:
        no_new += 1
        continue

    # Обрабатываем порциями, чтобы можно было остановить
    chunk_size = 100
    added_total = 0
    earliest_in_batch = None
    reached_pre_2025 = False

    for i in range(0, len(new_blocks), chunk_size):
        chunk = new_blocks[i:i + chunk_size]
        added, earliest, early_stop = collect_articles_from_blocks(chunk, stop_on_pre_2025=True)
        added_total += added

        if earliest:
            if earliest_in_batch is None or earliest < earliest_in_batch:
                earliest_in_batch = earliest

        if early_stop:
            reached_pre_2025 = True
            break

        # Не блочьте, пожалуйста
        time.sleep(0.1)

    total_scrolls += 1
    # print(f"Скролл {total_scrolls}: всего статей {len(articles_data)} (+{added_total})")

    # Предостарожность, чтобы не влипнуть на бесконечной прокрутке
    if earliest_in_batch and earliest_in_batch < CUTOFF_DATE:
        reached_pre_2025 = True

    if reached_pre_2025:
        print("Достигнута дата 1 января 2025.")
        break
driver.quit()
# Проверка для себя
"""
if articles_data:
    dates = [item["date"] for item in articles_data if item["date"]]
    earliest = min(dates) if dates else "неизвестно"
    latest = max(dates) if dates else "неизвестно"
    print(f"\nВсего собрано: {len(articles_data)} новостей")
    print(f"Диапазон дат: {earliest} — {latest}")
"""

# Парсинг BeautifulSoup
final_data = []
session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0 Safari/537.36"
})

for idx, item in enumerate(articles_data, 1):
    title = item['title']
    link = item['link']

    if is_weather_related(title, link):
        print(f"[{idx}/{len(articles_data)}] Пропущено (погода): {title[:60]}...")
        continue

    print(f"[{idx}/{len(articles_data)}] Парсинг: {title[:60]}...")

    details = get_article_details(link, session)
    if not details:
        continue

    # Повторно проверяем дату
    if details['date']:
        try:
            dt_obj = datetime.fromisoformat(details['date'].replace('Z', '+00:00'))
            if dt_obj.year != 2025:
                # print(f"Пропущено: не 2025 год ({dt_obj.year})")
                continue
        except Exception as e:
            print(f"Не удалось распарсить дату: {details['date']}")
    else:
        print("Дата не найдена")

    final_data.append({
        'title': title,
        'link': link,
        'date': details['date'],
        'lead': details['lead'],
        'text': details['text'],
        'tags': ', '.join(details['tags']) if details['tags'] else ''
    })

    time.sleep(0.5)

# Сохраняемся
df = pd.DataFrame(final_data)

df.drop_duplicates(subset=['link'], inplace=True)

# Фильтрация по 2025 году (если дата есть)
if 'date' in df.columns and not df.empty:
    def extract_year(date_str):
        if not date_str:
            return None
        try:
            return datetime.fromisoformat(date_str.replace('Z', '+00:00')).year
        except:
            return None
    df['year'] = df['date'].apply(extract_year)
    df = df[df['year'] == 2025].drop(columns=['year'])

print(df.head())

df.to_csv("ria_tomsk_2025_news.csv", index=False, encoding='utf-8-sig')

Streaming output truncated to the last 5000 lines.
[634/5600] Парсинг: Руководители томских больниц проведут открытый прием в понед...
[635/5600] Парсинг: Театры Томска и Северска покажут капустники на фестивале "Ма...
[636/5600] Парсинг: Силовики провели рейд на вечеринке "Шабаш" в Томске...
[637/5600] Пропущено (погода): МЧС предупреждает о резком похолодании в Томской области в п...
[638/5600] Парсинг: Как делать покупки в Черную пятницу: советы Роспотребнадзора...
[639/5600] Парсинг: Hyundai столкнулся с "Ладой" на трассе Томск – Мариинск...
[640/5600] Парсинг: Шесть человек из "ВАЗа" с томскими номерами погибли на трасс...
[641/5600] Парсинг: Капремонт идет в томском кадетском корпусе, работающем уже 2...
[642/5600] Парсинг: Томск – город, который устал: интервью с "главным мемологом"...
[643/5600] Парсинг: Четыре ленд-арт объекта появятся в 2026 году в ботсаду ТГУ...
[644/5600] Пропущено (погода): Мокрый снег и плюсовая температура ждут томичей в воскресень...
[645/5600] Парсинг: